In [1]:
import os
import cv2
import random
import numpy as np
import albumentations as A
import yaml

BASE_DIR = "/gpfs/gibbs/project/dan_wu/hz454/Peng_Liu/CelebA_subset_new"
AUG_DIR  = "/gpfs/gibbs/project/dan_wu/hz454/Peng_Liu/augmented_new"
OUT_DIR  = "/gpfs/gibbs/project/dan_wu/hz454/Peng_Liu/celeba_dataset_mosaic_new"
FINAL_W, FINAL_H = 640, 640
AUG_PER_IMG = 5
NUM_MOSAIC = 200 
TRAIN_RATIO = 0.8


transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=15, p=0.5),
    A.ColorJitter(p=0.3)
])

os.makedirs(AUG_DIR, exist_ok=True)
id_folders = [d for d in os.listdir(BASE_DIR) if d.startswith("images__")]
ids = {}

for i, folder in enumerate(sorted(id_folders)):
    ids[folder] = i  
    in_dir = os.path.join(BASE_DIR, folder)
    out_dir = os.path.join(AUG_DIR, folder.replace("images__", "id"))
    os.makedirs(out_dir, exist_ok=True)
    for img_name in os.listdir(in_dir):
        img_path = os.path.join(in_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        cv2.imwrite(os.path.join(out_dir, img_name), img)
        for j in range(AUG_PER_IMG):
            aug = transform(image=img)["image"]
            new_name = f"{os.path.splitext(img_name)[0]}_aug{j+1}.jpg"
            cv2.imwrite(os.path.join(out_dir, new_name), aug)
print(f" Step 1: Total {len(id_folders)} IDs")

/gpfs/gibbs/project/dan_wu/hz454/celeba_env/lib/python3.11/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


 Step 1: Total 46 IDs


In [27]:
for sub in ["images/train", "images/val", "labels/train", "labels/val"]:
    os.makedirs(os.path.join(OUT_DIR, sub), exist_ok=True)


# Step 2: define IoU
def iou(box1, box2):
    # box = (x1, y1, x2, y2)
    xi1, yi1 = max(box1[0], box2[0]), max(box1[1], box2[1])
    xi2, yi2 = min(box1[2], box2[2]), min(box1[3], box2[3])
    inter = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - inter + 1e-9
    return inter / union


# Step 3: place images（with IoU checking）
def place_image(canvas, img, cls_id, existing_boxes):
    H, W, _ = canvas.shape
    h, w, _ = img.shape

    max_scale = min(W / w, H / h, 0.8)
    min_scale = 0.5
    if max_scale <= min_scale:
        return None

    scale = random.uniform(min_scale, max_scale)
    new_w, new_h = int(w * scale), int(h * scale)
    img = cv2.resize(img, (new_w, new_h))

  
    for _ in range(30):
        x1 = random.randint(0, max(0, W - new_w))
        y1 = random.randint(0, max(0, H - new_h))
        x2, y2 = x1 + new_w, y1 + new_h
        new_box = (x1, y1, x2, y2)

        
        if all(iou(new_box, b) <= 0.05 for b in existing_boxes):
            canvas[y1:y2, x1:x2] = img
            existing_boxes.append(new_box)

            #  YOLO labels
            x_center = (x1 + x2) / 2 / W
            y_center = (y1 + y2) / 2 / H
            bw = (x2 - x1) / W
            bh = (y2 - y1) / H
            return f"{cls_id} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n"

    
    return None


# Step 4: loading images
img_dict = {}
for k in ids.keys():
    folder = os.path.join(AUG_DIR, k.replace('images__', 'id'))
    if not os.path.exists(folder):
        continue
    imgs = [os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith(".jpg")]
    if len(imgs) > 0:
        img_dict[k] = imgs


# Step 5: Mosaic
for i in range(NUM_MOSAIC):
    split = "train" if i < NUM_MOSAIC * TRAIN_RATIO else "val"
    canvas = np.ones((FINAL_H, FINAL_W, 3), dtype=np.uint8) * 255 
    num_faces = random.choice([3, 4]) 
    chosen_ids = random.choices(list(img_dict.keys()), k=num_faces)

    labels = []
    existing_boxes = [] 

    for id_key in chosen_ids:
        img_file = random.choice(img_dict[id_key])
        img = cv2.imread(img_file)
        if img is None:
            continue

        lbl = place_image(canvas, img, ids[id_key], existing_boxes)
        if lbl is not None:
            labels.append(lbl)

    out_img = os.path.join(OUT_DIR, f"images/{split}/mosaic_{i:04d}.jpg")
    out_lbl = os.path.join(OUT_DIR, f"labels/{split}/mosaic_{i:04d}.txt")
    cv2.imwrite(out_img, canvas)
    with open(out_lbl, "w") as f:
        f.writelines(labels)

print(f"✅ Step 2: Mosaic datasets generated, total {NUM_MOSAIC}")

✅ Step 2: Mosaic datasets generated, total 200


In [28]:
yaml_path = os.path.join(OUT_DIR, "data_mosaic.yaml")
yaml_data = {
    "path": OUT_DIR,
    "train": "images/train",
    "val": "images/val",
    "nc": len(ids),
    "names": list(ids.keys())
}
with open(yaml_path, "w") as f:
    yaml.dump(yaml_data, f)
print(f" Step 3: data_mosaic.yaml generate in {yaml_path}")
print(f" Datasets preparation finfished，total {len(ids)}")

 Step 3: data_mosaic.yaml generate in /gpfs/gibbs/project/dan_wu/hz454/Peng_Liu/celeba_dataset_mosaic_new/data_mosaic.yaml
 Datasets preparation finfished，total 46


In [29]:
from ultralytics import YOLO
model = YOLO("yolov8m.pt")
model.train(
    data="/gpfs/gibbs/project/dan_wu/hz454/Peng_Liu/celeba_dataset_mosaic_new/data_mosaic.yaml",
    epochs=200,
    imgsz=640,
    batch=64,
    project="/gpfs/gibbs/project/dan_wu/hz454/Peng_Liu/runs",
    name="celeb_mosaic_new"
)


New https://pypi.org/project/ultralytics/8.3.205 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.204 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA RTX A5000, 24123MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/gpfs/gibbs/project/dan_wu/hz454/Peng_Liu/celeba_dataset_mosaic_new/data_mosaic.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0

/gpfs/gibbs/project/dan_wu/hz454/celeba_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Plotting labels to /gpfs/gibbs/project/dan_wu/hz454/Peng_Liu/runs/celeb_mosaic_new7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.0002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /gpfs/gibbs/project/dan_wu/hz454/Peng_Liu/runs/celeb_mosaic_new7
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      1/200      12.9G     0.6945      4.552      1.018        176        640: 100% ━━━━━━━━━━━━ 3/3 1.5it/s 1.9s0.9s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 4.9it/s 0.2s
                   all         40        136     0.0107      0.373     0.0348     0.0302

      Epoch    GPU_mem   box_los

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x154ebdcc4d50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    

In [37]:
import os
import random
from ultralytics import YOLO


OUT_DIR = "/gpfs/gibbs/project/dan_wu/hz454/Peng_Liu/celeba_dataset_mosaic_new"
model_path = "runs/detect/train/weights/best.pt"


model = YOLO(model_path)


val_dir = os.path.join(OUT_DIR, "images/val")
label_dir = os.path.join(OUT_DIR, "labels/val")
all_imgs = [f for f in os.listdir(val_dir) if f.endswith(".jpg")]


sample_imgs = random.sample(all_imgs, 10)
print("Random images：")
for f in sample_imgs:
    print("  ", f)

correct = 0
total = 0


for img_name in sample_imgs:
    img_path = os.path.join(val_dir, img_name)
    label_path = os.path.join(label_dir, img_name.replace(".jpg", ".txt"))


    results = model.predict(source=img_path, conf=0.25, imgsz=640, verbose=False)
    boxes = results[0].boxes

 
    if not os.path.exists(label_path):

        continue
    with open(label_path) as f:
        gt_lines = [line.strip().split() for line in f.readlines()]
    gt_ids = [int(l[0]) for l in gt_lines]

   
    pred_ids = [int(b.cls[0].item()) for b in boxes]

  
    matched = sum(1 for pid in pred_ids if pid in gt_ids)
    total += len(gt_ids)
    correct += matched

    print(f"\n{img_name}")
    print(f"real ID: {gt_ids}")
    print(f"distinguish ID: {pred_ids}")
    for b in boxes:
        x1, y1, x2, y2 = b.xyxy[0].tolist()
        conf = float(b.conf[0].item())
        cls = int(b.cls[0].item())
        print(f"   -> ID {cls} @ ({x1:.0f},{y1:.0f},{x2:.0f},{y2:.0f}) conf={conf:.2f}")


acc = correct / total if total > 0 else 0
print(f"\n precision rate: {acc:.2%} （{correct}/{total}）")


Random images：
   mosaic_0171.jpg
   mosaic_0197.jpg
   mosaic_0162.jpg
   mosaic_0196.jpg
   mosaic_0164.jpg
   mosaic_0179.jpg
   mosaic_0166.jpg
   mosaic_0190.jpg
   mosaic_0183.jpg
   mosaic_0188.jpg

mosaic_0171.jpg
real ID: [17, 35, 0, 17]
distinguish ID: [17, 0, 35, 0, 18]
   -> ID 17 @ (258,64,355,183) conf=1.00
   -> ID 0 @ (506,211,630,363) conf=0.99
   -> ID 35 @ (394,391,522,547) conf=0.90
   -> ID 0 @ (117,369,251,534) conf=0.60
   -> ID 18 @ (116,369,252,534) conf=0.26

mosaic_0197.jpg
real ID: [15, 21, 42]
distinguish ID: [15, 42]
   -> ID 15 @ (377,381,512,546) conf=1.00
   -> ID 42 @ (90,371,221,532) conf=0.96

mosaic_0162.jpg
real ID: [3, 35, 33, 14]
distinguish ID: [35, 11, 40, 7]
   -> ID 35 @ (41,43,133,156) conf=1.00
   -> ID 11 @ (33,319,141,452) conf=0.95
   -> ID 40 @ (209,193,333,343) conf=0.81
   -> ID 7 @ (173,481,270,600) conf=0.29

mosaic_0196.jpg
real ID: [40, 42, 29, 6]
distinguish ID: [40, 29, 22, 13]
   -> ID 40 @ (173,67,304,230) conf=0.97
   -> ID 2